In [1]:
import ntpath
import re

import numpy as np
import pandas as pd

In [2]:
def read_datadoc(path):
    f = open(path, 'rb')

    nonnull_fields = []
    for line in f:
        fields = line.decode('cp932').strip().split()
        if fields and len(fields) > 1:
            nonnull_fields.append(fields)

    startpos = None
    for i, line in enumerate(nonnull_fields):
        if line[0] == '項目名':
            startpos = i + 1
            break

    endpos = None
    for i, line in enumerate(nonnull_fields[startpos:], startpos):
        if '**' in line[0]:
            endpos = i
            break

    template_fields = [field for field in nonnull_fields[startpos:endpos] if len(field) > 3]
    for field in template_fields:
        colname = field[0]
        print(field)

read_datadoc('kta_doc.txt')

['場コード', '2', '99', '1']
['年', '2', '99', '3']
['回', '1', '9', '5']
['日', '1', 'F', '6', '16進数(数字', 'or', '小文字アルファベット)']
['Ｒ', '2', '99', '7']
['馬名キー', '40', 'X', '9']
['血統登録番号', '8', 'X', '49']
['馬名', '36', 'X', '57', '全角１８文字']
['性別コード', '1', '9', '93', '1:牡,2:牝,3,セン']
['馬記号コード', '2', '99', '94', 'コード表参照']
['ブリンカー', '1', 'X', '96', '1:初装着,2:再装着,3:ブリンカ']
['騎手名', '12', 'X', '97', '全角６文字']
['負担重量', '3', '999', '109', '0.1Kg単位']
['見習い区分', '1', '9', '112', '1:☆(1K減),2:△(2K),3:▲(3K)']
['調教師名', '12', 'X', '113', '全角６文字']
['調教師所属', '4', 'X', '125', '全角２文字']
['ＩＤＭ', '5', 'ZZ9.9', '129']
['上昇度', '1', '9', '134']
['ローテーション', '3', 'ZZ9', '135', '間に金曜日が入っている数で決定']
['脚質', '1', '9', '138']
['距離適性', '1', '9', '139']
['距離適性２', '1', '9', '140']
['芝適性コード', '1', 'X', '141', '1:◎,', '2:○,', '3:△']
['ダ適性コード', '1', 'X', '142', '1:◎,', '2:○,', '3:△']
['重適性コード', '1', '9', '143', '1:◎,', '2:○,', '3:△']
['蹄コード', '2', '99', '144']
['クラスコード', '2', '99', '146']
['毛色コード', '2', '99', '148']
['前走１競走成績キー', '16', '9', 

In [7]:
COLNAMES = ['colname', 'OCC', 'width', 'type', 'rel_pos', 'comment']

templates = {
    'KTA': {
        'name': 'JRDB登録馬データ',
        'config': [
            ['場コード', None, '2', '99', '1'],
            ['年', None, '2', '99', '3'],
            ['回', None, '1', '9', '5'],
            ['日', None, '1', 'F', '6', '16進数(数字 or 小文字アルファベット)'],
            ['Ｒ', None, '2', '99', '7'],
            ['馬名キー', None, '40', 'X', '9'],
            ['血統登録番号', None, '8', 'X', '49'],
            ['馬名', None, '36', 'X', '57', '全角１８文字'],
            ['性別コード', None, '1', '9', '93', '1:牡,2:牝,3,セン'],
            ['馬記号コード', None, '2', '99', '94', 'コード表参照'],
            ['ブリンカー', None, '1', 'X', '96', '1:初装着,2:再装着,3:ブリンカ'],
            ['騎手名', None, '12', 'X', '97', '全角６文字'],
            ['負担重量', None, '3', '999', '109', '0.1Kg単位'],
            ['見習い区分', None, '1', '9', '112', '1:☆(1K減),2:△(2K),3:▲(3K)'],
            ['調教師名', None, '12', 'X', '113', '全角６文字'],
            ['調教師所属', None, '4', 'X', '125', '全角２文字'],
            ['ＩＤＭ', None, '5', 'ZZ9.9', '129'],
            ['上昇度', None, '1', '9', '134'],
            ['ローテーション', None, '3', 'ZZ9', '135', '間に金曜日が入っている数で決定'],
            ['脚質', None, '1', '9', '138'],
            ['距離適性', None, '1', '9', '139'],
            ['距離適性２', None, '1', '9', '140'],
            ['芝適性コード', None, '1', 'X', '141', '1:◎, 2:○, 3:△'],
            ['ダ適性コード', None, '1', 'X', '142', '1:◎, 2:○, 3:△'],
            ['重適性コード', None, '1', '9', '143', '1:◎, 2:○, 3:△'],
            ['蹄コード', None, '2', '99', '144'],
            ['クラスコード', None, '2', '99', '146'],
            ['毛色コード', None, '2', '99', '148'],
            ['前走１競走成績キー', None, '16', '9', '150'],
            ['前走２競走成績キー', None, '16', '9', '166'],
            ['前走３競走成績キー', None, '16', '9', '182'],
            ['前走４競走成績キー', None, '16', '9', '198'],
            ['前走５競走成績キー', None, '16', '9', '214'],
            ['前走１レースキー', None, '8', '9', '230'],
            ['前走２レースキー', None, '8', '9', '238'],
            ['前走３レースキー', None, '8', '9', '246'],
            ['前走４レースキー', None, '8', '9', '254'],
            ['前走５レースキー', None, '8', '9', '262'],
            ['騎手コード', None, '5', '9', '270', '騎手マスタとリンク'],
            ['調教師コード', None, '5', '9', '275', '調教師マスタとリンク'],
            ['獲得賞金', None, '6', 'ZZZZZ9', '280', '単位万円(含む付加賞)'],
            ['収得賞金', None, '5', 'ZZZZ9', '286', '単位万円'],
            ['条件クラス', None, '1', '9', '291', '条件グループコード参照'],
            ['走法', None, '8', '9', '292', 'コード表参照'],
            ['体型', None, '24', 'X', '300', 'コード表参照'],
            ['体型総合１', None, '3', '9', '324', '特記コード参照'],
            ['体型総合２', None, '3', '9', '327', '特記コード参照'],
            ['体型総合３', None, '3', '9', '330', '特記コード参照'],
            ['馬特記１', None, '3', '9', '333', '特記コード参照'],
            ['馬特記２', None, '3', '9', '336', '特記コード参照'],
            ['馬特記３', None, '3', '9', '339', '特記コード参照'],
            ['テン指数', None, '5', 'ZZ9.9', '342', '予想テン指数'],
            ['ペース指数', None, '5', 'ZZ9.9', '347', '予想ペース指数'],
            ['上がり指数', None, '5', 'ZZ9.9', '352', '予想上がり指数'],
            ['位置指数', None, '5', 'ZZ9.9', '357', '予想位置指数'],
            ['馬スタート指数', None, '4', 'Z9.9', '362'],
            ['馬出遅率', None, '4', 'Z9.9', '366'],
            ['参考前走', None, '2', '99', '370', '参考となる前走（２走分格納）'],
            ['参考前走騎手コード', None, '5', 'X', '372', '参考となる前走の騎手'],
            ['データ区分', None, '1', 'X', '377', '1:特別登録, 2:想定確定'],
            ['出走順位', None, '3', 'X', '378', '参考データ※1'],
            ['予備', None, '6', 'X', '381', 'スペース'],
            ['改行', None, '2', 'X', '387', 'ＣＲ・ＬＦ']
        ]
     }
}

class JRDBReader:
    def __init__(self, path):
        # things like db connection (needed for to_sql call)
        self.path = path

    def get_template(self):
        for key, template in templates.items():
            filename = ntpath.basename(self.path)
            match = re.search('([A-Za-z]+)[0-9]+', filename)
            if match:
                match_key = match.group(1)
                if key.lower() == match_key.lower():
                    df = pd.DataFrame(template['config'], columns=COLNAMES)
                    df.width = df.width.astype(np.int64)
                    df.rel_pos = df.rel_pos.astype(np.int64)
                    df.name = template['name']
                    return df
        raise ValueError(f'template not found for file: {file}')
    
    def read(self):
        template = self.get_template()
        template_len = len(template.index)
        with open(self.path, 'rb') as f:
            rows = []
            for line in f:
                if line == b'\n':
                    continue
                row = []
                for i in range(template_len):
                    config = template.iloc[i]
                    start = config.rel_pos - 1
                    stop = start + config.width
                    cell = line[start:stop].decode('cp932')
                    row.append(cell)
                rows.append(row)
        return pd.DataFrame(rows, columns=template.colname)
    
    def save(self):
        # pd.to_sql
        pass

In [9]:
df = JRDBReader('./KTA040627.txt').read()

In [11]:
df['脚質']

0      0
1      2
2      3
3      2
4      5
5      5
6      4
7      2
8      0
9      5
10     2
11     0
12     3
13     2
14     3
15     2
16     3
17     2
18     4
19     4
20     3
21     4
22     5
23     5
24     1
25     2
26     5
27     3
28     3
29     4
      ..
442    3
443    2
444    5
445    3
446    2
447    3
448    2
449    2
450    3
451    3
452    2
453    4
454    5
455    3
456    3
457    3
458    3
459    2
460    3
461    2
462    2
463    4
464    4
465    2
466    5
467    3
468    1
469    1
470    4
471    4
Name: 脚質, Length: 472, dtype: object